In [1]:
import sys
import pandas as pd
import numpy as np
import warnings
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from load_data import LoadData

In [2]:
loader = LoadData()

In [3]:
telco_df = loader.read_excel('../data/Week1_challenge_data_source.xlsx')

In [8]:
#sys.path.append(os.path.abspath(os.path.join('../tests')))
#import test_load_data
%run ../tests/test_load_data.py

.
----------------------------------------------------------------------
Ran 1 test in 220.610s

OK


In [4]:
pd = loader.read_excel('../data/Week1_challenge_data_source.xlsx')